In [2]:
!pip install pytorch-pretrained-bert


     |████████████████████████████████| 133kB 2.9MB/s 
     |████████████████████████████████| 645kB 8.8MB/s 


In [0]:
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim
from nltk.tokenize import TweetTokenizer
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
import string
import os
from os import path
import tqdm
from IPython.display import display, clear_output
from sklearn.model_selection import train_test_split
import wandb

In [61]:
wandb.init(project="NLP-HW3")

W&B Run: https://app.wandb.ai/psolikov/NLP-HW3/runs/rb68md0b

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
DRIVE_PATH = 'drive/My Drive/Colab Notebooks/'
INPUT_PATH = 'input.txt'
OUTPUT_PATH = 'output.txt'
TRAIN_PATH = 'Collection3/'

In [6]:
tokenizer = TweetTokenizer()
puncs = set(string.punctuation)
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased',do_lower_case=False)
train_labels_dict = {
    "LOC":0,
    "PER":1,
    "ORG":2
}

100%|██████████| 995526/995526 [00:01<00:00, 922791.37B/s]


In [0]:
class TrainPreToken:
  def __init__(self, file, text, index, length, label, name, parsed_name):
    self.file = file
    self.text = text
    self.index = index
    self.length = length
    self.label = label
    self.name = name
    self.parsed_name = parsed_name

  def __str__(self):
    return f"file: {self.file} index: {self.index} length: {self.length} label: {self.label} name: {self.name} parsed_name: {self.parsed_name}"

In [0]:
def load_input_data():
  with open(DRIVE_PATH + INPUT_PATH) as f:
    return f.readlines()

def load_train_data_files():
  train_txts = []
  train_anns = []
  for file in os.listdir(DRIVE_PATH + TRAIN_PATH):
    if file.endswith(".txt"):
      filename = os.path.splitext(file)[0]
      if path.exists(DRIVE_PATH + TRAIN_PATH + filename +  '.ann'):
        train_txts.append(file)
        train_anns.append(filename +  '.ann')
      else:
        print(DRIVE_PATH + TRAIN_PATH + filename +  '.ann')
  train_txts = sorted(train_txts)
  train_anns = sorted(train_anns)
  return train_txts, train_anns

def extract_train_token(train_token):
  train_token = train_token.split('\t')
  token_name = train_token[0]
  token_info = train_token[1]
  token_str = train_token[2]
  token_class, token_start, token_end = token_info.split()
  return token_name, token_str, token_class, token_start, token_end

def train_text_to_bert_pre_tokens(file, train_text, train_tokens):
  bert_pre_tokens = []
  current_text = train_text
  for train_token in train_tokens:
    token_name, token_str, token_class, token_start, token_end = extract_train_token(train_token)
    token_start = int(token_start)
    token_end = int(token_end)
    left_start = len(train_text) - len(current_text)
    pre_token_left = TrainPreToken(file, train_text, left_start, token_start - left_start, 0, 'left', train_text[left_start:token_start])
    bert_pre_tokens.append(pre_token_left)
    pre_token_found  = TrainPreToken(file, train_text, token_start, token_end - token_start, train_labels_dict[token_class], token_name, train_text[token_start:token_end])
    bert_pre_tokens.append(pre_token_found)
    current_text = train_text[token_end:]
    # print(bert_pre_tokens)
  right_start = len(train_text) - len(current_text)
  pre_token_right = TrainPreToken(file, train_text, right_start, len(train_text) - right_start, 0, 'right', train_text[right_start:])
  bert_pre_tokens.append(pre_token_right)
  return bert_pre_tokens

def load_train_data():
  train_txts, train_anns = load_train_data_files()
  bert_tokens_per_file = []
  itr = 0
  for train_txt, train_ann in tqdm.tqdm(zip(train_txts, train_anns)):
    clear_output(wait=True)
    print(itr)
    itr += 1
    train_text = ""
    train_tokens = []
    with open(DRIVE_PATH + TRAIN_PATH + train_txt) as f:
      train_text = f.read().replace('\n\n', '\n\n\n\n')
    with open(DRIVE_PATH + TRAIN_PATH + train_ann) as f:
      for line in f.readlines():
        train_tokens.append(line)
    # print(train_text, train_tokens)
    bert_pre_tokens = train_text_to_bert_pre_tokens(train_txt, train_text, train_tokens)
    bert_tokens_per_file.append(bert_pre_tokens)
    # break
  return bert_tokens_per_file

def add_index_and_length(tokens, text):
  index = 0
  result = []
  for token in tokens:
    result.append((token, index, len(token)))
    add_space = 1 if index + len(token) < len(text) and text[index + len(token)] == ' ' else 0
    index += len(token) + add_space
  return result

def tokenize_lines(lines):
  lines_tokens = [add_index_and_length(tokenizer.tokenize(x), x) for x in lines]
  return lines_tokens

def mark_text(lines_tokens):
  return [[]]

def first_elements_tup(tokens):
  return list(map(lambda x: x[0], tokens))

In [0]:
def prepare_train_data(bert_tokens_per_file):
  train_data = []
  for bert_tokens in bert_tokens_per_file:
    for pre_token in bert_tokens:
      label = pre_token.label
      bert_subtokens = bert_tokenizer.tokenize(pre_token.parsed_name)
      # bert_subtokens = ["[CLS]"] + bert_subtokens + ["[SEP]"]
      for bert_subtoken in bert_subtokens:
        train_data.append([bert_subtoken, label])
  return train_data

In [0]:
class WordBertClassifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc = nn.Linear(768, 3)

  def forward(self, x):
    z = self.fc(x)
    return z

In [11]:
bert_tokens_per_file = load_train_data()

999


1000it [22:13,  1.40s/it]


In [0]:
train_data = np.array(prepare_train_data(bert_tokens_per_file))
train_data_x = train_data[:,0]
train_data_y = train_data[:,1]

In [0]:
x_train, x_test, y_train, y_test = train_test_split(train_data_x, train_data_y, test_size=0.1)
train_data_all = list(zip(x_train, y_train))
test_data_all = list(zip(x_test, y_test))

In [0]:
train_data_loader = data.DataLoader(dataset=train_data_all, batch_size=256, shuffle=True)
test_data_loader = data.DataLoader(dataset=test_data_all, batch_size=256, shuffle=False)

In [0]:
model = WordBertClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), 0.01)

In [0]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)
bert_model = BertModel.from_pretrained('bert-base-multilingual-cased')

In [85]:
train_data_all[:10]

[('.', '0'),
 ('##а', '0'),
 ('р', '0'),
 ('##ал', '0'),
 ('##у', '0'),
 ('##ом', '2'),
 ('.', '1'),
 ('ок', '0'),
 ('##ж', '0'),
 ('успешно', '0')]

In [82]:
n_epochs = 40
for epoch in range(n_epochs):
  train_losses = []
  test_losses = []
  for batch in train_data_loader:
    tokens = batch[0]
    labels = batch[1]
    indexed_tokens = bert_tokenizer.convert_tokens_to_ids(tokens)
    segments_ids = [1] * len(tokens)

    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    with torch.no_grad():
      encoded_layers, _ = bert_model(tokens_tensor, segments_tensors)

    batch_i = 0
    token_i = 5
    layer_i = 5
    token_embeddings = [] 
    for token_i in range(len(tokens)):    
      hidden_layers = [] 
      for layer_i in range(len(encoded_layers)):
        vec = encoded_layers[layer_i][batch_i][token_i]
        hidden_layers.append(vec)
      token_embeddings.append(hidden_layers)

    summed_last_4_layers = [torch.sum(torch.stack(layer)[-4:], 0) for layer in token_embeddings] # [number_of_tokens, 768]

    #####

    optimizer.zero_grad()
    output = model(torch.stack(summed_last_4_layers).to(device))
    loss = criterion(output, torch.from_numpy(np.array(list(map(lambda x: int(x), list(labels))))).to(device))
    train_loss = loss.item()
    clear_output(wait=True)
    train_losses.append(train_loss)
    print(f"Epoch: {epoch} Train loss: {train_loss}")
    loss.backward()
    optimizer.step()

    # break

  train_loss = np.mean(train_losses)
  wandb.log({'train_loss': train_loss})

  for idx_batch, batch in enumerate(test_data_loader):
    tokens = batch[0]
    labels = batch[1]
    indexed_tokens = bert_tokenizer.convert_tokens_to_ids(tokens)
    segments_ids = [1] * len(tokens)

    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    with torch.no_grad():
      encoded_layers, _ = bert_model(tokens_tensor, segments_tensors)

    batch_i = 0
    token_i = 5
    layer_i = 5
    token_embeddings = [] 
    for token_i in range(len(tokens)):    
      hidden_layers = [] 
      for layer_i in range(len(encoded_layers)):
        vec = encoded_layers[layer_i][batch_i][token_i]
        hidden_layers.append(vec)
      token_embeddings.append(hidden_layers)

    summed_last_4_layers = [torch.sum(torch.stack(layer)[-4:], 0) for layer in token_embeddings] # [number_of_tokens, 768]

    # print('here')

    with torch.no_grad():
      optimizer.zero_grad()
      output = model(torch.stack(summed_last_4_layers).to(device))
      loss = criterion(output, torch.from_numpy(np.array(list(map(lambda x: int(x), list(labels))))).to(device))
      test_loss = loss.item()
      test_losses.append(test_loss)
      clear_output(wait=True)
      print(f"Epoch: {epoch} batch: {idx_batch} Test loss: {test_loss}")

  test_loss = np.mean(test_losses)
  wandb.log({'test_loss': test_loss})

Epoch: 5 Train loss: 0.8171058297157288


KeyboardInterrupt: ignored

In [0]:
indexed_tokens = bert_tokenizer.convert_tokens_to_ids(tmp[0])

In [0]:
indexed_tokens
segments_ids = [1] * len(tmp[0])

In [0]:
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])
with torch.no_grad():
    encoded_layers, _ = bert_model(tokens_tensor, segments_tensors)

In [0]:
batch_i = 0

token_i = 5
layer_i = 5

token_embeddings = [] 

# For each token in the sentence...
for token_i in range(len(tmp[0])):
  
  # Holds 12 layers of hidden states for each token 
  hidden_layers = [] 
  
  # For each of the 12 layers...
  for layer_i in range(len(encoded_layers)):
    
    # Lookup the vector for `token_i` in `layer_i`
    vec = encoded_layers[layer_i][batch_i][token_i]
    
    hidden_layers.append(vec)
    
  token_embeddings.append(hidden_layers)



In [76]:
summed_last_4_layers = [torch.sum(torch.stack(layer)[-4:], 0) for layer in token_embeddings] # [number_of_tokens, 768]

256

In [77]:
len(summed_last_4_layers[0])

768